In [ ]:
import pyodbc
import pandas as pd

# Kapcsolati string
server = 'bixsql01.bixpert.local'  # pl. 'localhost' vagy '192.168.1.100'
database = 'JDS_Verseny_DEV'
username = 'sa'
password = 'Galeria1'
driver = '{ODBC Driver 17 for SQL Server}'  # Győződj meg róla, hogy telepítve van

# Csatlakozás létrehozása
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

query = "SELECT  * FROM L_COUNTRY"

df = pd.read_sql(query, conn)

# Kapcsolat lezárása
conn.close()

/var/folders/h8/jsx_20090y338vjhdy9lwbnc0000gn/T/ipykernel_87297/3006382410.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [22]:
df.head(100)

,COUNTRYID,COUNTRY_CODE,COUNTRYDESC,COUNTRYLONGDESC,COUNTRY_SORTID,COUNTRY_VALID
0,1,N/A,Not_Defined,None,1,1
1,2,-1,-1,None,1,1
2,3,10,10,None,1,1
3,4,11,11,None,1,1
4,5,12,12,None,1,1
...,...,...,...,...,...,...
95,96,99,99,None,1,1
96,97,AD,AD,None,1,1
97,98,AE,AE,None,1,1
98,99,AF,AF,None,1,1
